## turn categorical data into numerical

In [2]:

from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
df = pd.read_csv('heart-2020-cleaned-numerical.csv')


In [3]:
print(len(df))

319795


In [4]:
df = df.replace('Yes', 1, regex=True).replace('No', 0, regex=True).replace('Yes (during pregnancy)', 1, regex=True)
df = df.replace('Female', 0, regex=True).replace('Male', 1, regex=True)
df = df.replace('18-24', 0, regex=True).replace("25-29", 1, regex=True).replace("30-34", 2, regex=True).replace("35-39", 3, regex=True)
df = df.replace("40-44", 4, regex=True).replace("45-49", 5, regex=True).replace("50-54", 6, regex=True).replace("55-59", 7, regex=True)
df = df.replace("60-64", 8, regex=True).replace("65-69", 9, regex=True).replace("70-74", 10, regex=True).replace("75-79", 11, regex=True).replace("80 or older", 12, regex=True)

df =df.replace("American Indian/Alaskan Native", 0, regex=True)
df =df.replace("Asian", 1, regex=True)
df =df.replace("Black", 2, regex=True)
df =df.replace("Hispanic", 3, regex=True)
df =df.replace("White", 4, regex=True)
df =df.replace("Other", 5, regex=True)



df.head(5)

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,...,AmericanIndian/AlaskanNative,Diabetic,DiabeticWhilePregnant,BorderlineDiabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,0,16.60,1,0,0,3.0,30.0,0,1,7,...,0,1,0,0,1,4,5.0,1,0,1
1,0,20.34,0,0,1,0.0,0.0,0,1,12,...,0,0,0,0,1,4,7.0,0,0,0
2,0,26.58,1,0,0,20.0,30.0,0,0,9,...,0,1,0,0,1,2,8.0,1,0,0
3,0,24.21,0,0,0,0.0,0.0,0,1,11,...,0,0,0,0,0,3,6.0,0,0,1
4,0,23.71,0,0,0,28.0,0.0,1,1,4,...,0,0,0,0,1,4,8.0,0,0,0


## balance data

In [5]:
# seperate by col value
df_HD = df.loc[df['HeartDisease'] == 1]
df_NHD = df.loc[df['HeartDisease'] == 0]

#get equal amount of sample of each
df_HD = df_HD.sample(27373)
df_NHD = df_NHD.sample(27373)
# merge the two types
df = pd.concat([df_HD, df_NHD], axis=0)


## split into testing and training data

In [6]:
X = df.loc[:, df.columns != 'HeartDisease']
y = df['HeartDisease']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)


## Random Forest Classifier Training

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

In [8]:
# basic model
rfclf = RandomForestClassifier()
cv_results = cross_validate(rfclf, X_train, y_train, cv=5)
print('random forest test score: ', sum(cv_results['test_score'])/len(cv_results['test_score']))

random forest test score:  0.7478995277395044


In [9]:
#param tuning
from sklearn.model_selection import GridSearchCV

parameters = {
    'max_depth' : np.arange(0,11, 1), 
    'min_samples_split': np.arange(0, 11, 1)
    # 'subsample': np.arange(0, 1.1, 0.1)
}
rf = GridSearchCV(rfclf, parameters,cv=5, scoring='f1', verbose=1, n_jobs=-1)
rf.fit(X_train,y_train)

Fitting 5 folds for each of 121 candidates, totalling 605 fits


C:\Users\miera\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan 0.73971839 0.73598944 0.74055637 0.72944485 0.73597891
 0.74225056 0.73974472 0.73647545 0.74139944        nan        nan
 0.74916764 0.74332767 0.7478682  0.74745713 0.74972833 0.74966221
 0.74804913 0.74984721 0.74907415        nan        nan 0.75274163
 0.75462933 0.75321943 0.75335608 0.75596377 0.75334713 0.75444664
 0.75368843 0.75236282        nan        nan 0.75925897 0.75881999
 0.75566199 0.75769641 0.7622111  0.75677112 0.75882825 0.75686817
 0.75808667        nan        nan 0.76179717 0.76169359 0.76304671
 0.760205   0.7658424  0.76352444 0.76077033 0.76442853 0.76122001
        nan        nan 0.77077836 0.7703168  0.76788432 0.76955957
 0.7666366  0.77059338 0.76844152 0.76936875 0.770427

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                         'min_samples_split': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])},
             scoring='f1', verbose=1)

In [9]:
print('rf test score: ', rf.best_score_)
print('rf best params', rf.best_params_)

rf test score:  0.7783873561931195
rf best params {'max_depth': 9, 'min_samples_split': 8}


In [10]:
rfclf2 = RandomForestClassifier()
cv_results = cross_validate(rfclf2, X_train, y_train, cv=5)
print('random forest test score: ', sum(cv_results['test_score'])/len(cv_results['test_score']))

random forest test score:  0.7442916353225202


In [11]:
# further param tuning
parameters = {
    'max_depth' : np.arange(5,15, 1), 
    'min_samples_split': np.arange(8, 15, 1),
    'criterion': ['gini', 'entropy', 'log_loss']
}
rf2 = GridSearchCV(rfclf2, parameters,cv=5, scoring='f1', verbose=1, n_jobs=-1)
rf2.fit(X_train,y_train)

Fitting 5 folds for each of 210 candidates, totalling 1050 fits


C:\Users\miera\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [0.76773782 0.76573699 0.76803531 0.7684105  0.76661179 0.767091
 0.76766955 0.77304273 0.7729229  0.77063199 0.77097623 0.7721425
 0.77350469 0.77235356 0.77465447 0.77459331 0.77503228 0.77510224
 0.77520971 0.77376032 0.77644184 0.77736354 0.77746554 0.77718108
 0.77638182 0.77712488 0.77717055 0.77668138 0.77798107 0.77740342
 0.77787144 0.7784808  0.7774338  0.77706044 0.77730936 0.77800822
 0.77728169 0.77800382 0.77707355 0.77839178 0.77766639 0.77821013
 0.77685552 0.7774886  0.77666864 0.77705139 0.77674822 0.77791822
 0.77740307 0.77722848 0.77694036 0.77605247 0.77759858 0.77788482
 0.77733781 0.77754587 0.77694435 0.77709379 0.77709359 0.77668682
 0.77744498 0.77721293 0.77704685 0.77512404 0.77615997 0.77659114
 0.77649544 0.77646018 0.77775736 0.77691907 0.7664774  0.76824267
 0.76638659 0.76262168 0.76590064 0.76618889 0.76434059 

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
                         'min_samples_split': array([ 8,  9, 10, 11, 12, 13, 14])},
             scoring='f1', verbose=1)

In [12]:
print('rf test score: ', rf2.best_score_)
print('rf best params', rf2.best_params_)
# rf test score:  0.7784807991221412
# rf best params {'criterion': 'gini', 'max_depth': 9, 'min_samples_split': 11}

rf test score:  0.7784807991221412
rf best params {'criterion': 'gini', 'max_depth': 9, 'min_samples_split': 11}


In [12]:
rfclf3 = RandomForestClassifier()
cv_results3 = cross_validate(rfclf3, X_train, y_train, cv=5)
print('random forest test score: ', sum(cv_results3['test_score'])/len(cv_results3['test_score']))

random forest test score:  0.7427619236782415


In [14]:
# third round of tuning
parameters = {
    'max_depth' : [9], 
    'min_samples_split': np.arange(8, 12, 1),
    'criterion': ['gini', 'entropy', 'log_loss'],
    'n_estimators': np.arange(100, 200, 10),
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': ['True', 'False'],
    
}
rf3 = GridSearchCV(rfclf3, parameters,cv=5, scoring='f1', verbose=1, n_jobs=-1)
rf3.fit(X_train,y_train)

Fitting 5 folds for each of 720 candidates, totalling 3600 fits


C:\Users\miera\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [0.77616318 0.7763963  0.77609613 0.77611268 0.77653663 0.77595294
 0.7764135  0.77620982 0.77689699 0.77739718 0.77657481 0.77665505
 0.77643575 0.77660413 0.77665292 0.77590903 0.77720668 0.77704272
 0.77681444 0.77644593 0.7762734  0.77679242 0.77728618 0.77663479
 0.77680468 0.77576558 0.77619871 0.7764763  0.77598192 0.7762198
 0.77710672 0.77650712 0.77642233 0.77688614 0.77763792 0.776049
 0.77702976 0.77705729 0.77743784 0.77726572 0.77636633 0.77635294
 0.77628816 0.77654798 0.77628837 0.77616524 0.77653314 0.77652623
 0.77683364 0.77606658 0.7768064  0.77620518 0.77675957 0.77667423
 0.77669975 0.77724008 0.77655095 0.77747865 0.77646571 0.77702427
 0.77688063 0.77668577 0.77672682 0.77719305 0.77681308 0.77681818
 0.77637431 0.7760033  0.77703014 0.77738352 0.77681394 0.7769357
 0.77716133 0.7764997  0.77652863 0.77627146 0.77660062 0

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': ['True', 'False'],
                         'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [9],
                         'max_features': ['sqrt', 'log2', None],
                         'min_samples_split': array([ 8,  9, 10, 11]),
                         'n_estimators': array([100, 110, 120, 130, 140, 150, 160, 170, 180, 190])},
             scoring='f1', verbose=1)

In [15]:
print('rf test score: ', rf3.best_score_)
print('rf best params', rf3.best_params_)
# rf test score:  0.777637919551298
# rf best params {'bootstrap': 'True', 'criterion': 'gini', 'max_depth': 9, 'max_features': 'sqrt', 'min_samples_split': 11, 'n_estimators': 140}

rf test score:  0.777637919551298
rf best params {'bootstrap': 'True', 'criterion': 'gini', 'max_depth': 9, 'max_features': 'sqrt', 'min_samples_split': 11, 'n_estimators': 140}


## Extra Trees Classifier

In [8]:
from sklearn.ensemble import ExtraTreesClassifier

In [13]:
# initial model
etclf = ExtraTreesClassifier()
cv_results2 = cross_validate(etclf, X_train, y_train, cv=5)
print('random forest test score: ', sum(cv_results2['test_score'])/len(cv_results2['test_score']))

random forest test score:  0.7278518351118042


In [14]:
# tuning
parameters = {
    'max_depth' : np.arange(1,10, 1), 
    'min_samples_split': np.arange(8, 15, 1),
    'criterion': ['gini', 'entropy', 'log_loss']
}
et = GridSearchCV(etclf, parameters,cv=5, scoring='f1', verbose=1, n_jobs=-1)
et.fit(X_train,y_train)

Fitting 5 folds for each of 189 candidates, totalling 945 fits


C:\Users\miera\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [0.69176132 0.69578681 0.70940749 0.68847132 0.70881577 0.70875078
 0.6986199  0.71793476 0.7081678  0.72688617 0.71601243 0.72435725
 0.718137   0.71157187 0.73240021 0.73397622 0.73943627 0.73369025
 0.71962558 0.73727326 0.73221034 0.74756854 0.74397212 0.74978745
 0.74604077 0.74997972 0.74350137 0.74559475 0.75653615 0.75109672
 0.75571144 0.75466463 0.75523318 0.75553263 0.75391219 0.76200241
 0.76055615 0.75840014 0.76173126 0.76022752 0.7600807  0.75973089
 0.76172091 0.76501754 0.76399027 0.76479026 0.76316951 0.76379187
 0.76394476 0.76695406 0.76794146 0.76927585 0.76762734 0.76652528
 0.76771838 0.76846325 0.77010944 0.76901223 0.77053854 0.77001234
 0.76878099 0.77051017 0.76850753 0.70986777 0.69356424 0.70174931
 0.69793274 0.70586826 0.70668544 0.70889766 0.72031444 0.71556458
 0.71391251 0.72201399 0.71418265 0.72296    0.719602

GridSearchCV(cv=5, estimator=ExtraTreesClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
                         'min_samples_split': array([ 8,  9, 10, 11, 12, 13, 14])},
             scoring='f1', verbose=1)

In [15]:
print('rf test score: ', et.best_score_)
print('rf best params', et.best_params_)
# rf test score:  0.7723301951842264
# rf best params {'criterion': 'entropy', 'max_depth': 9, 'min_samples_split': 12}

rf test score:  0.7723301951842264
rf best params {'criterion': 'entropy', 'max_depth': 9, 'min_samples_split': 12}


In [16]:
etclf2 = ExtraTreesClassifier()
cv_results4 = cross_validate(etclf2, X_train, y_train, cv=5)
print('random forest test score: ', sum(cv_results4['test_score'])/len(cv_results4['test_score']))

random forest test score:  0.7273722918266456


In [17]:
# further tuning
parameters = {
    'max_depth' : np.arange(9,19, 1), 
    'min_samples_split': np.arange(11, 18, 1),
    'criterion': ['entropy'],
    'max_features': ['sqrt', 'log2', None]
    
}
et2 = GridSearchCV(etclf2, parameters,cv=5, scoring='f1', verbose=1, n_jobs=-1)
et2.fit(X_train,y_train)

Fitting 5 folds for each of 210 candidates, totalling 1050 fits


GridSearchCV(cv=5, estimator=ExtraTreesClassifier(), n_jobs=-1,
             param_grid={'criterion': ['entropy'],
                         'max_depth': array([ 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]),
                         'max_features': ['sqrt', 'log2', None],
                         'min_samples_split': array([11, 12, 13, 14, 15, 16, 17])},
             scoring='f1', verbose=1)

In [19]:
print('rf test score: ', et2.best_score_)
print('rf best params', et2.best_params_)

rf test score:  0.7754580458308484
rf best params {'criterion': 'entropy', 'max_depth': 12, 'max_features': 'log2', 'min_samples_split': 16}


In [21]:
# more tuning
etclf3 = ExtraTreesClassifier()
parameters = {
    'max_depth' : [12], 
    'min_samples_split': np.arange(11, 25, 1),
    'criterion': ['entropy'],
    'max_features': ['log2'],
    'bootstrap': ['true', 'false'],
    'oob_score': ['true', 'false'],
    'random_state':  np.arange(1, 5, 1)
    
}
et3 = GridSearchCV(etclf3, parameters,cv=5, scoring='f1', verbose=1, n_jobs=-1)
et3.fit(X_train,y_train)

Fitting 5 folds for each of 224 candidates, totalling 1120 fits


GridSearchCV(cv=5, estimator=ExtraTreesClassifier(), n_jobs=-1,
             param_grid={'bootstrap': ['true', 'false'],
                         'criterion': ['entropy'], 'max_depth': [12],
                         'max_features': ['log2'],
                         'min_samples_split': array([11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]),
                         'oob_score': ['true', 'false'],
                         'random_state': array([1, 2, 3, 4])},
             scoring='f1', verbose=1)

In [22]:
print('rf test score: ', et3.best_score_)
print('rf best params', et3.best_params_)

rf test score:  0.7761679592501389
rf best params {'bootstrap': 'true', 'criterion': 'entropy', 'max_depth': 12, 'max_features': 'log2', 'min_samples_split': 24, 'oob_score': 'true', 'random_state': 3}


In [9]:
# additional tuning
etclf4 = ExtraTreesClassifier()
parameters = {
    'max_depth' : [12], 
    'min_samples_split': np.arange(23, 28, 1),
    'min_samples_leaf': np.arange(1, 10, 1),
    'criterion': ['entropy'],
    'max_features': ['log2'],
    'bootstrap': ['true', 'false'],
    'oob_score': ['true', 'false'],
    'random_state':  np.arange(1, 5, 1)    
}
et4 = GridSearchCV(etclf4, parameters,cv=5, scoring='f1', verbose=1, n_jobs=-1)
et4.fit(X_train,y_train)

Fitting 5 folds for each of 720 candidates, totalling 3600 fits


GridSearchCV(cv=5, estimator=ExtraTreesClassifier(), n_jobs=-1,
             param_grid={'bootstrap': ['true', 'false'],
                         'criterion': ['entropy'], 'max_depth': [12],
                         'max_features': ['log2'],
                         'min_samples_leaf': array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
                         'min_samples_split': array([23, 24, 25, 26, 27]),
                         'oob_score': ['true', 'false'],
                         'random_state': array([1, 2, 3, 4])},
             scoring='f1', verbose=1)

In [10]:
print('rf test score: ', et4.best_score_)
print('rf best params', et4.best_params_)

# rf test score:  0.7737919732320606
# rf best params {'bootstrap': 'true', 'criterion': 'entropy', 'max_depth': 12, 'max_features': 'log2', 'min_samples_leaf': 4, 'min_samples_split': 25, 'oob_score': 'true', 'random_state': 3}

rf test score:  0.7737919732320606
rf best params {'bootstrap': 'true', 'criterion': 'entropy', 'max_depth': 12, 'max_features': 'log2', 'min_samples_leaf': 4, 'min_samples_split': 25, 'oob_score': 'true', 'random_state': 3}
